## Location Tracking Status
A guide showing how to quickly view some details of the location tracking capability in your organization

This guide assumes that:
1. You are an administrator for your organization
2. You are using Enterprise 10.7+
3. You are running this with the latest version of the ArcGIS API for Python

In [1]:
from arcgis.gis import GIS

gis = GIS("https://server.domain.com/webadapter", "admin", "password", verfiy_cert=False)
lt = gis.admin.location_tracking

### Check Current Status of Location Tracking
Let's quickly check to see the status of location tracking for our organization

In [2]:
print(f"Status: {lt.status}")
print(f"Retention Period: {lt.retention_period} {lt.retention_period_units} ({'enabled' if lt.retention_period_enabled else 'disabled'})")

Status: enabled
Retention Period: 30 DAYS (disabled)


### Check Licenses
Let's see how many licenses have been assigned

In [3]:
gis.admin.license.get('Tracker for ArcGIS').report

,Entitlement,Total,Assigned,Remaining
0,tracker,200,129,71


### Check Active Users
Let's see how many people have actually recorded tracks

In [4]:
users = lt.tracks_layer.query(group_by_fields_for_statistics="created_user", 
                              out_statistics=[{"statisticType": "count", "onStatisticField": "objectid", "outStatisticFieldName": "count"}],
                              order_by="count",
                              as_df=True)
print(f"Users: {len(users)}")

Users: 148


Let's see who the top 5 users are (based on how many tracks points they have uploaded)

In [5]:
users.head(5)

,count,created_user
0,536228,apulver
1,424272,cgillgrass
2,383522,tmorey
3,267023,jshaner
4,247850,jwhitney


Let's see who the top 5 users were during the last week

In [6]:
from datetime import datetime, timedelta
d = datetime.utcnow()-timedelta(days=7)
users = lt.tracks_layer.query(where=f"location_timestamp >= timestamp '{d.strftime('%Y-%m-%d %H:%M:%S')}'",
                              group_by_fields_for_statistics="created_user", 
                              out_statistics=[{"statisticType": "count", "onStatisticField": "objectid", "outStatisticFieldName": "count"}],
                              order_by="count",
                              as_df=True)
users.head(5)

,count,created_user
0,32894,apulver
1,19445,jshaner
2,9470,cgillgrass
3,9244,bszukalski
4,8282,igomez


### Check total number of tracks
Let's see how many tracks are available

In [7]:
count = lt.tracks_layer.query(return_count_only=True)
print(f"Total Tracks: {count:,}")

Total Tracks: 4,629,564


### Check number of track views
Let's see how many track views are in my organization

In [8]:
items = gis.content.search("typekeywords:'Location Tracking View'")
print(f"Track Views: {len(items)}")

Track Views: 10
